In [1]:
from src.localdataset.dataset import LocalDataset
from cordata.devices.caed.ecg import ECG
from pathlib import Path
from functools import cache
import random

Using cordata.corpulsweb: 2023.11.6
Using cordata.devices: 2023.11.6
Using cordata.lake: 2023.11.6


In [2]:
ds = LocalDataset("/media/HDD01/mbck/dataSet/")

In [3]:
@cache
def cached_loader(path:Path):
    return (ECG.from_files(str(path) + ".log")).all()["value_in_microvolt"].to_numpy()


def not_cached_loader(path:Path):
    return (ECG.from_files(str(path) + ".log")).all()["value_in_microvolt"].to_numpy()

Ohne cache

In [4]:
split = ds.get_data("split", preprocessing="window_300", dataset_version=7, loader=not_cached_loader).train

In [5]:
%timeit -n 100 split.get_by_index(random.randint(0, len(split))).data

955 ms ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
del(split)

Mit cache

In [7]:
cached_split = ds.get_data("split", preprocessing="window_300", dataset_version=7, loader=cached_loader).train

In [8]:
%timeit -n 100 cached_split.get_by_index(random.randint(0, len(cached_split))).data

The slowest run took 11.76 times longer than the fastest. This could mean that an intermediate result is being cached.
230 ms ± 211 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
del(cached_split)

## Grouped_by GID

Ohne Chache

In [12]:
gid_grouped = ds.get_data_sorted_by_base_gid(preprocessing="window_300", dataset_version=7, loader=not_cached_loader).train

In [13]:
%timeit -n 100 gid_grouped.get_by_index(random.randint(0, len(gid_grouped)-1)).get_next_samples(500)

434 ms ± 31.4 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
del(gid_grouped)

mit Cache

In [15]:
cached_gid_grouped = ds.get_data_sorted_by_base_gid(preprocessing="window_300", dataset_version=7, loader=cached_loader).train

In [16]:
%timeit -n 100 cached_gid_grouped.get_by_index(random.randint(0, len(cached_gid_grouped)-1)).get_next_samples(500)

235 ms ± 69.2 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
del(cached_gid_grouped)